In [10]:
from PIL import Image
import pandas as pd
import os
import glob
import shutil
import json
import re
from sklearn.model_selection import train_test_split

seed = 42

In [5]:
#cargando json con el encoding de labels a un diccionario
ruta_json='../dataset/labelsFineTuning.json'
with open(ruta_json, 'r') as f:
      labels_dict = json.load(f)

In [6]:
ruta_json='../raw/Medellin2/anotaciones/'
rows = []

for archivo in os.listdir(ruta_json):
      if archivo.endswith('.json'):
            ruta_archivo = os.path.join(ruta_json, archivo)
            with open(ruta_archivo, 'r', encoding='utf-8') as f:
                  data = json.load(f)
                  image_key = data.get("key")
                  img_width = data.get("width")
                  img_height = data.get("height")

                  match = re.search(r'_(\d+)\.', image_key)
                  frame = int(match.group(1)) if match else None


                  for box in data.get("boxes", []):
                        row = {
                              "file": image_key,
                              "frame": frame,
                              "image_width": img_width,
                              "image_height": img_height,
                              "id": box.get("id"),
                              "label": box.get("label"),
                              "x": float(box.get("x")),
                              "y": float(box.get("y")),
                              "w": float(box.get("width")),
                              "h": float(box.get("height")),
                              "confidence": box.get("confidence")
                        }
                        rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)

In [7]:
df

,file,frame,image_width,image_height,id,label,x,y,w,h,confidence
0,frame_0000.jpg,0,1920,1080,1,DHelmet,251.45,568.60,66.99,87.08,None
1,frame_0000.jpg,0,1920,1080,2,DHelmetP1Helmet,402.17,616.78,72.65,89.66,None
2,frame_0000.jpg,0,1920,1080,3,DHelmet,737.61,800.47,82.96,99.45,None
3,frame_0000.jpg,0,1920,1080,4,DHelmet,1137.46,936.25,85.02,99.96,None
4,frame_0000.jpg,0,1920,1080,5,DHelmet,1309.82,1007.87,93.78,113.36,None
...,...,...,...,...,...,...,...,...,...,...,...
825,frame_0154.jpg,154,1920,1080,1,DHelmet,192.85,706.25,109.08,113.85,None
826,frame_0154.jpg,154,1920,1080,2,DHelmet,453.47,795.28,109.23,124.17,None
827,frame_0154.jpg,154,1920,1080,3,DHelmet,464.28,868.50,127.27,122.49,None
828,frame_0154.jpg,154,1920,1080,4,DHelmetP1Helmet,758.26,880.85,110.26,127.55,None


In [8]:
img_width = df["image_width"].max()
img_height = df["image_height"].max()
print(f"Images width: {img_width}, Images height: {img_height}")

Images width: 1920, Images height: 1080


In [11]:
df_imagenes_unicas = df[['file']].drop_duplicates().reset_index(drop=True)
print(f"Imagenes unicas: {len(df_imagenes_unicas)}")

indices_imagenes = df_imagenes_unicas.index

train_indices, test_indices = train_test_split(
    indices_imagenes,
    test_size=0.3,
    random_state=seed
)

df_imagenes_train = df_imagenes_unicas.loc[train_indices].reset_index(drop=True)
df_imagenes_test = df_imagenes_unicas.loc[test_indices].reset_index(drop=True)

print(f"Número de imágenes para entrenamiento: {len(df_imagenes_train)}")
print(f"Número de imágenes para prueba: {len(df_imagenes_test)}")

Imagenes unicas: 126
Número de imágenes para entrenamiento: 88
Número de imágenes para prueba: 38


In [13]:
#copiando imagenes test a su respectiva carpeta
ruta_origen=r'../raw/Medellin2'
ruta_destino_imagenes=r'../dataset/images/test'
ruta_destino_anotaciones=r'../dataset/labels/test'
image_width=img_width
image_height=img_height
target_size=(640, 640)


for index, row in df_imagenes_test.iterrows():
    #copiando imagen
    file = row['file']
    frame_jpg = row['file'].split('.')[0]
    ruta_imagen_origen = os.path.join(ruta_origen,'imagenes', f'{file}')
    ruta_imagen_destino = os.path.join(ruta_destino_imagenes, f'{file}')
    #shutil.copy(ruta_imagen_origen, ruta_imagen_destino)

    try:
            img = Image.open(ruta_imagen_origen)
            img_resized = img.resize(target_size)
            img_resized.save(ruta_imagen_destino)
    except FileNotFoundError:
        print(f"Warning: Image not found at {ruta_imagen_origen}. Skipping.")
        continue # Saltar a la siguiente imagen si no se encuentra

    #generando anotaciones
    txt_filename = f"{frame_jpg}.txt"
    txt_filepath = os.path.join(ruta_destino_anotaciones, txt_filename)
    for index, row in df[(df.file==file)].iterrows():
        class_id=labels_dict[row['label']]
        #esto era si x y y estaban en esquina y no en centrol
        #x_center = int(row['x']+row['w']/2)
        #y_center = int(row['y']+row['h']/2)

        x_center = row['x']
        y_center = row['y']


        x_center_norm = x_center / image_width
        y_center_norm = y_center / image_height
        width_norm = row['w'] / image_width
        height_norm = row['h'] / image_height

        yolov8_annotation=f"{class_id} {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}\n"

        with open(txt_filepath, 'a') as f:
            f.write(yolov8_annotation)
